In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Input.xlsx')

In [3]:
output_folder = 'output_folder/'

Data Extraction

In [4]:
import requests
from bs4 import BeautifulSoup
import os


def sanitize_filename(filename):
    invalid_chars = '/\\?%*:|"<>!'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

def extract_article_content(url, output_folder='output/'):

    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.title.text.strip()

    sanitized_title = sanitize_filename(title)

    body = soup.body
    inner_div = None

    if body:
        outer_wrap = body.find('div', {'id': 'td-outer-wrap', 'class': 'td-theme-wrap'})
        if outer_wrap:
            article = outer_wrap.find('article')
            if article:
                td_containers = article.find_all('div', {'class': 'td-container'})
                if td_containers:
                    inner_div = td_containers[-1]

    if inner_div:
        inner_div2 = inner_div.find('div', {'class': 'td-pb-row'})
        if inner_div2:
            inner_div3 = inner_div2.find('div', {'class': 'td-pb-span8 td-main-content', 'role': 'main'})
            inner_div4 = inner_div3.find('div', {'class': 'td-ss-main-content'})
            inner_div5 = inner_div4.find('div', {'class': 'td-post-content tagdiv-type'})

            if inner_div5:
                inner_div5_text = inner_div5.get_text(separator='\n', strip=True)

                # output folder
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                url_id = None
                if df is not None:
                    url_id_series = df[df['URL'] == url]['URL_ID']
                    if not url_id_series.empty:
                        url_id = url_id_series.iloc[0]

                # Save content with URL_ID
                if url_id:
                    filename = f'{url_id}.txt'
                    file_path = os.path.join(output_folder, filename)
                    with open(file_path, 'w', encoding='utf-8') as content_file:
                        content_file.write(f"Title: {title}\n\n{inner_div5_text}")

                    print(f"Content extracted for {url} with URL_ID {url_id}")
                    return

    print(f"No content extracted for {url}")



In [5]:
for index, row in df.iterrows():
    url = row['URL']
    extract_article_content(url, output_folder)

Content extracted for https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/ with URL_ID blackassign0001
Content extracted for https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/ with URL_ID blackassign0002
Content extracted for https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/ with URL_ID blackassign0003
Content extracted for https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/ with URL_ID blackassign0004
Content extracted for https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/ with URL_ID blackassign0005
Content extracted for https://insights.blackcoffer.com/the-rise-of-the-ott-platform-and-its-impact-on-the-entertainment-industry-by-2040/ with URL_ID blackassign0006
Content

Data Sentiment Analysis

In [6]:
pip install nltk

In [7]:
import os
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [8]:
def filter_stopwords(words, stopwords):
    return [word for word in words if word.lower() not in stopwords]

def calculate_positive_score(text, positive_words):
    return sum(1 for word in text.split() if word.lower() in positive_words)

def calculate_negative_score(text, negative_words):
    return sum(1 for word in text.split() if word.lower() in negative_words)

def calculate_polarity_score(positive_score, negative_score):
    denominator = (positive_score + negative_score) + 1e-6
    return (positive_score - negative_score) / denominator

def calculate_subjectivity_score(positive_score, negative_score, total_words):
    denominator = (total_words + 1e-6)
    return (positive_score + negative_score) / denominator

def average_word_length(words):
    total_length = sum(len(word) for word in words)
    return total_length / len(words) if len(words) > 0 else 0


In [9]:
def load_stopwords_from_files(stopwords_files, encoding='utf-8'):
    combined_stopwords = set()
    for file_path in stopwords_files:
        with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
            stopwords_list = set(file.read().splitlines())
        combined_stopwords.update(stopwords_list)
    return combined_stopwords

def load_word_dictionary(file_path, encoding='latin-1'):
    with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
        word_list = file.read().splitlines()
    return set(word_list)


In [10]:
stopwords_files = ['StopWords_Auditor.txt', 'StopWords_Currencies.txt', 'StopWords_DatesandNumbers.txt',
                   'StopWords_Generic.txt', 'StopWords_GenericLong.txt', 'StopWords_Geographic.txt', 'StopWords_Names.txt']

combined_stopwords = load_stopwords_from_files(stopwords_files)

In [11]:
positive_words_file = 'positive-words.txt'
negative_words_file = 'negative-words.txt'
positive_words = load_word_dictionary(positive_words_file)
negative_words = load_word_dictionary(negative_words_file)

In [12]:
output_folder = 'output_folder/'

In [13]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import cmudict
nltk.download('punkt')
nltk.download('cmudict')
import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [14]:
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 802.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.6 MB/s eta 0:00:00


In [15]:

output_df = pd.DataFrame(columns=[
    'URL_ID',
    'URL',
    'POSITIVE SCORE',
    'NEGATIVE SCORE',
    'POLARITY SCORE',
    'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH',
    'PERCENTAGE OF COMPLEX WORDS',
    'FOG INDEX',
    'AVG NUMBER OF WORDS PER SENTENCE',
    'COMPLEX WORD COUNT',
    'WORD COUNTt',
    'SYLLABLE PER WORD',
    'PERSONAL PRONOUNS',
    'AVG WORD LENGTH'
])

In [16]:
from textstat import syllable_count

In [17]:
pip install natsort

In [18]:
from natsort import natsorted


for filename in natsorted(os.listdir(output_folder)):
    if filename.endswith('.txt'):
        file_path = os.path.join(output_folder, filename)
        print(f"Processing file: {filename}")
        url_id, _ = os.path.splitext(filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            article_text = file.read()

            #sentiment analysis
            tokens = word_tokenize(article_text)
            filtered_tokens = filter_stopwords(tokens, combined_stopwords)
            filtered_text = ' '.join(filtered_tokens)
            positive_score = calculate_positive_score(filtered_text, positive_words)
            negative_score = calculate_negative_score(filtered_text, negative_words)
            total_words = len(filtered_tokens)
            polarity_score = calculate_polarity_score(positive_score, negative_score)
            subjectivity_score = calculate_subjectivity_score(positive_score, negative_score, total_words)

            #additional variables
            sentences = sent_tokenize(article_text)
            avg_sentence_length = len(tokens) / len(sentences)

            #textstat for syllable counting
            complex_words = [word for word in tokens if syllable_count(word) > 2]

            percentage_complex_words = (len(complex_words) / len(tokens)) * 100
            fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

            avg_number_words_per_sentence = total_words / len(sentences)
            complex_word_count = len(complex_words)
            word_count = len(tokens)
            syllable_per_word = sum(syllable_count(word) for word in tokens) / len(tokens)

            #personal pronouns count using regex
            personal_pronouns_count = len(re.findall(r'\b(I|we|my|ours|us)\b', article_text, flags=re.IGNORECASE))

            avg_word_length = average_word_length(tokens)


            url = df[df['URL_ID'] == url_id]['URL'].iloc[0]

            row_data = {
                'URL_ID': url_id,
                'URL': url,
                'POSITIVE SCORE': positive_score,
                'NEGATIVE SCORE': negative_score,
                'POLARITY SCORE': polarity_score,
                'SUBJECTIVITY SCORE': subjectivity_score,
                'AVG SENTENCE LENGTH': avg_sentence_length,
                'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
                'FOG INDEX': fog_index,
                'AVG NUMBER OF WORDS PER SENTENCE': avg_number_words_per_sentence,
                'COMPLEX WORD COUNT': complex_word_count,
                'WORD COUNTt': word_count,
                'SYLLABLE PER WORD': syllable_per_word,
                'PERSONAL PRONOUNS': personal_pronouns_count,
                'AVG WORD LENGTH': avg_word_length
}

            output_df = pd.concat([output_df, pd.DataFrame([row_data])], ignore_index=True)



output_df.to_csv('output_results1.csv', index=False)


Processing file: blackassign0001.txt
Processing file: blackassign0002.txt
Processing file: blackassign0003.txt
Processing file: blackassign0004.txt
Processing file: blackassign0005.txt
Processing file: blackassign0006.txt
Processing file: blackassign0007.txt
Processing file: blackassign0008.txt
Processing file: blackassign0009.txt
Processing file: blackassign0010.txt
Processing file: blackassign0011.txt
Processing file: blackassign0012.txt
Processing file: blackassign0013.txt
Processing file: blackassign0015.txt
Processing file: blackassign0016.txt
Processing file: blackassign0017.txt
Processing file: blackassign0018.txt
Processing file: blackassign0019.txt
Processing file: blackassign0021.txt
Processing file: blackassign0022.txt
Processing file: blackassign0023.txt
Processing file: blackassign0024.txt
Processing file: blackassign0025.txt
Processing file: blackassign0026.txt
Processing file: blackassign0027.txt
Processing file: blackassign0028.txt
Processing file: blackassign0030.txt
P

In [19]:
from google.colab import files

output_csv_path = 'output_results1.csv'

files.download(output_csv_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>